In [4]:
import pandas as pd
import glob, os, re
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import scipy.io as spio
from scipy import signal, fft
from collections import defaultdict
from datetime import datetime, timedelta
from obspy import read, Trace, UTCDateTime
import numpy.matlib
from matplotlib import cm
from scipy.io.wavfile import read as wavread
import warnings
warnings.filterwarnings("ignore")
def to_pandas(data_set):
    datadict = {}
    for strg in data_set['detec']._fieldnames:
        elem = data_set['detec'].__dict__[strg]
        if isinstance(elem, spio.matlab.mat_struct):
            datadict[strg] = _todict(elem)
        else:
            datadict[strg] = elem
    if isinstance(elem, np.ndarray):
        return pd.DataFrame(datadict)
    else:
        return pd.DataFrame(datadict, index=[0])
def getinfo(place, year, index):
    df = pd.read_csv('../DataBase/Experiment_period.txt')
    df_ = df[df['region']==place][df['year']==year][df['index']==index]
    S = -1*df_['h_calib'].values[0]
    begintime = pd.to_datetime(df_['begin'].values[0])
    endtime = pd.to_datetime(df_['end'].values[-1])
    # month = str(begintime.month).zfill(2)
    return begintime, endtime, S
def readwav(path, S):
    def calib(data, S):
        return data / ((10)**(S/20))
    rate, data = wavread(path)
    data = data.astype(np.float32, order='C') / 32768.0
    data = calib(data, S)
    
    tr = Trace()
    tr.data = data
    tr.stats.sampling_rate = rate
    date = path.rsplit('.')[1]
    
    tr.stats.starttime = UTCDateTime(f'20{date[:10]}00')
    # print(trslice.stats.statttime)
    tr = tr.filter('bandpass', freqmin=100, freqmax=2000, corners=4, zerophase=True)
    return tr
def findpeakf(tr):
    try:
        freq = np.linspace(0, tr.stats.sampling_rate/2, tr.stats.npts // 2 + 1)
        spec = abs(fft.rfft(tr.data))
        maxidx = np.argmax(spec)
        peakf = freq[maxidx]
        return peakf
    except:
        return np.nan
    

def calc_SPL(tr):
    try:
        p_rms = np.sqrt(np.mean(tr.data**2))
        SPL = 20*np.log10(p_rms/1)
        return SPL
    except:
        return np.nan

In [5]:
path = '/Volumes/home/Research/Detect_fish/Taoyuan_csv'
begin, end, S = getinfo('Taoyuan', 2023, 2)
wavdir = '/Volumes/Data/Soundscape/03_TaoYuan/2023/20230608-20230623/wav'
savepath = '/Volumes/home/Research/Detect_fish/Taoyuan_fishcall_grunt'

In [6]:
for d in range(13,16):
    current = begin+timedelta(days=d)
    print(current)

2023-06-21 08:00:00
2023-06-22 08:00:00
2023-06-23 08:00:00


In [7]:
for d in range(0,3):
    current = begin+timedelta(days=d)
    print(current)
    yy = str(current.year)[2:4]
    mm = str(current.month).zfill(2)
    dd = str(current.day).zfill(2)
    daydf = pd.DataFrame()
    
    for dirpath in sorted(glob.glob(f'{path}/{yy}{mm}{dd}*')):
        hhdf = pd.DataFrame()
        SPL_list = []
        peakf_list = []
        for file in glob.glob(f'{dirpath}/*.mat'):
            mat = spio.loadmat(file, struct_as_record=False, squeeze_me=True)
            df = to_pandas(mat)
            df_ = df[df['frequency_min']<=600][df['frequency_max']<=2000]
            hhdf = pd.concat([hhdf, df_]) 
            
        if len(hhdf)>0:
            print(dirpath, len(hhdf))
            globwav = glob.glob(f'{wavdir}/*{dirpath.rsplit("/")[-1]}*')[0]
            print(globwav)
            tr = readwav(globwav, S)
            for i in range(len(hhdf)):
                begintmp = re.split('[(|)|,]',hhdf['time_min_date'].values[i]); endtmp = re.split('[(|)|,]',hhdf['time_max_date'].values[i])
                tr_begin = UTCDateTime(int(begintmp[1]),int(begintmp[2]),int(begintmp[3]),int(begintmp[4]),int(begintmp[5]),int(begintmp[6]),int(begintmp[7]))
                tr_end = UTCDateTime(int(endtmp[1]),int(endtmp[2]),int(endtmp[3]),int(endtmp[4]),int(endtmp[5]),int(endtmp[6]),int(endtmp[7]))
                
                # globwav =  glob.glob(f'{wavdir}/*{str(tr_begin.day).zfill(2)}{str(tr_begin.hour).zfill(2)}{str(tr_begin.minute//5*5).zfill(2)}*.wav')[0]
                # print(globwav)
                # tr = readwav(globwav, S)
                
                tr_slice = tr.copy()
                tr_slice = tr_slice.slice(tr_begin, tr_end); del begintmp, endtmp

                
                SPL = calc_SPL(tr_slice)
                peak_f = findpeakf(tr_slice)
                SPL_list.append(SPL); peakf_list.append(peak_f)
                
                #--------- plot 
                fig, ax = plt.subplots(2,1,figsize=(6,4))
                plt.rc('font', family='Helvetica')
                ax[0].plot(tr_slice.times()*1000, tr_slice.data, 'k')
                ax[0].set_xlim(tr_slice.times()[0]*1000, tr_slice.times()[-1]*1000)
                try:
                    ax[0].set_title(f'{tr_slice.stats.starttime}\nSPL: {round(SPL,2)} dB, peak_f:{round(peak_f)} Hz')
                except:
                    ax[0].set_title(f'{tr_slice.stats.starttime}\nSPL: {SPL} dB, peak_f:{peak_f} Hz')
                ax[0].set_xlabel('Time (ms)')

                length = 96
                tr_slice_down = tr_slice.decimate(factor=5, no_filter=True)
                f,t,P = signal.stft(tr_slice_down.data, fs= tr_slice_down.stats.sampling_rate, window='hann', 
                      nperseg=length, noverlap=int(0.95*length), nfft=2000, detrend='constant', boundary=None, padded=False)
                data = 10*np.log10(np.abs(P)/np.power(1, 2)) #+ 176.2

                axis=1
                ambient = np.percentile(data, 10, axis=axis)
                if axis==0:
                    input_data = np.subtract(data, np.matlib.repmat(ambient, data.shape[axis], 1))
                elif axis==1:
                    input_data = np.subtract(data, np.matlib.repmat(ambient, data.shape[axis], 1).T)
                im1 = ax[1].imshow(input_data, vmin=6, vmax=12,
                           origin='lower',  aspect='auto', cmap=cm.jet,
                           extent=[t[0]*1000, t[-1]*1000, f[0], f[-1]], interpolation='none')
                ax[1].set_ylabel('Frequency (Hz)', fontsize=10)
                ax[1].set_xlabel('Time (ms)' , fontsize=10)
                ax[1].set_ylim(100, 2500)
                ax[1].set_xlim(tr_slice.times()[0]*1000, tr_slice.times()[-1]*1000)
                hour = str(tr_slice.stats.starttime.hour).zfill(2)
                minute = str(tr_slice.stats.starttime.minute).zfill(2)
                second = str(tr_slice.stats.starttime.second).zfill(2)
                if not os.path.exists(f'{savepath}/fig/23{mm}{dd}'):
                    os.makedirs(f'{savepath}/fig/23{mm}{dd}')
                    os.makedirs(f'{savepath}/msd/23{mm}{dd}')
                plt.savefig(f'{savepath}/fig/23{mm}{dd}/23{mm}{dd}_{hour}{minute}{second}.png', bbox_inches='tight')
                tr_slice.write(f'{savepath}/msd/23{mm}{dd}/23{mm}{dd}_{hour}{minute}{second}.mseed', format='MSEED') 
                plt.close()
        hhdf['SPL'] = SPL_list
        hhdf['peak_f'] = peakf_list
        daydf = pd.concat([daydf, hhdf], ignore_index=True)    
    daydf.to_csv(f'../Results/Detection_fishcall_grunt/csv/result_23{mm}{dd}.csv', index=False)
print('finish...')

2023-06-08 08:00:00
2023-06-09 08:00:00
/Volumes/home/Research/Detect_fish/Taoyuan_csv/23060917 27
/Volumes/Data/Soundscape/03_TaoYuan/2023/20230608-20230623/wav/6979.230609170001.wav
/Volumes/home/Research/Detect_fish/Taoyuan_csv/23060918 4
/Volumes/Data/Soundscape/03_TaoYuan/2023/20230608-20230623/wav/6979.230609180001.wav
/Volumes/home/Research/Detect_fish/Taoyuan_csv/23060919 8
/Volumes/Data/Soundscape/03_TaoYuan/2023/20230608-20230623/wav/6979.230609190001.wav
/Volumes/home/Research/Detect_fish/Taoyuan_csv/23060920 14
/Volumes/Data/Soundscape/03_TaoYuan/2023/20230608-20230623/wav/6979.230609200001.wav
/Volumes/home/Research/Detect_fish/Taoyuan_csv/23060921 27
/Volumes/Data/Soundscape/03_TaoYuan/2023/20230608-20230623/wav/6979.230609210001.wav
/Volumes/home/Research/Detect_fish/Taoyuan_csv/23060922 28
/Volumes/Data/Soundscape/03_TaoYuan/2023/20230608-20230623/wav/6979.230609220001.wav
/Volumes/home/Research/Detect_fish/Taoyuan_csv/23060923 22
/Volumes/Data/Soundscape/03_TaoYuan/202